
# CMSC 320 — Checkpoint 2: Data Preprocessing & EDA  
**Dataset:** Biodiversity by County — Distribution of Animals, Plants, and Natural Communities (New York)

This notebook fulfills **Checkpoint 2 (25 pts)**:
- **Data preprocessing (5 pts):** (a) import, (b) parse, (c) organize  
- **Basic data exploration & summary statistics (20 pts):** **three conclusions** using **three different statistical methods, including a hypothesis test**, with at least **one plot per method**.


In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import re

%matplotlib inline


## 1) Data Preprocessing — Import, Parse, Organize (5 pts)

In [ ]:

# (a) Import
csv_path = "Biodiversity_by_County_-_Distribution_of_Animals__Plants_and_Natural_Communities.csv"
df_raw = pd.read_csv(csv_path)

# Organize: Clean column names
def clean_cols(cols):
    return [re.sub(r'[^0-9a-zA-Z_]+', '', c.strip().lower().replace(' ', '_')) for c in cols]

df = df_raw.copy()
df.columns = clean_cols(df.columns)

# Normalize string fields
text_cols = ['county','category','taxonomic_group','taxonomic_subgroup','scientific_name','common_name',
             'ny_listing_status','federal_listing_status','state_conservation_rank','global_conservation_rank',
             'distribution_status']
for col in text_cols:
    df[col] = df[col].astype(str).str.strip()

# (b) Parse: year ranges into numeric start/end/mid; listing flags to ints; rarity scores to numeric
def parse_year_range(s):
    if not isinstance(s, str) or s.strip() == "":
        return pd.NA, pd.NA, pd.NA
    s = s.strip().lower()
    m_range = re.match(r'^(\d{4})\s*[-–]\s*(\d{4})$', s)
    if m_range:
        y1, y2 = int(m_range.group(1)), int(m_range.group(2))
        return y1, y2, int(round((y1 + y2) / 2))
    m_single = re.match(r'^(\d{4})$', s)
    if m_single:
        y = int(m_single.group(1))
        return y, y, y
    return pd.NA, pd.NA, pd.NA

yr_cols = df['year_last_documented'].apply(parse_year_range).tolist()
yr_df = pd.DataFrame(yr_cols, columns=['year_last_documented_start','year_last_documented_end','year_last_documented_mid'])
df = pd.concat([df.drop(columns=['year_last_documented']), yr_df], axis=1)

def s_rank_to_score(s):
    if not isinstance(s, str) or s.strip() == "":
        return np.nan
    s = s.upper()
    m = re.search(r'S([1-5])', s)
    if m:
        digit = int(m.group(1))
        return 6 - digit  # S1 -> 5 (rare) ... S5 -> 1 (common)
    if s.startswith('SH') or s.startswith('SX'):
        return 5
    return np.nan

NEG_LISTING = {"", "none", "n/a", "na", "not listed", "notlisted", "game with open season", "game", "open season"}
def listed_flag(x):
    return 0 if str(x).strip().lower() in NEG_LISTING else 1

df['state_rarity_score']   = df['state_conservation_rank'].apply(s_rank_to_score)
df['federally_listed_flag'] = df['federal_listing_status'].apply(listed_flag).astype(int)
df['state_listed_flag']     = df['ny_listing_status'].apply(listed_flag).astype(int)

# (c) Organize: one-record-per-(county,species); keep a tidy pandas DataFrame
df_unique = df.drop_duplicates(subset=['county','scientific_name']).copy()

# Quick dataset profile (meets "what are the main characteristics? how many features/entries?")
profile = {
    "n_rows_raw": len(df_raw),
    "n_rows_clean": len(df),
    "n_unique_species_records": len(df_unique),
    "n_counties": df_unique['county'].nunique(),
    "n_features": df_unique.shape[1],
    "columns": df_unique.columns.tolist()
}
profile


## 2) Basic Data Exploration & Summary Statistics (20 pts)


### Method 1 — Composition & Outliers (Descriptive Statistics)  
**Goal:** Identify **over-represented categories/taxa** and **outlier counties** in species richness.  
**Plot:** Bar charts (category composition; Top-15 counties).  


In [ ]:

# Category composition (over-representation)
cat_counts = df_unique['category'].value_counts().sort_values(ascending=False)

plt.figure()
cat_counts.plot(kind='bar')
plt.title("Composition by Category")
plt.xlabel("Category")
plt.ylabel("Record count (unique species per county)")
plt.xticks(rotation=0)
plt.tight_layout()
plt.show()

# Species richness by county
species_by_county = df_unique.groupby('county')['scientific_name'].nunique().sort_values(ascending=False)
desc = species_by_county.describe()
print(desc.to_dict())

# Outliers via IQR
Q1, Q3 = species_by_county.quantile(0.25), species_by_county.quantile(0.75)
IQR = Q3 - Q1
upper_fence = Q3 + 1.5*IQR
outliers = species_by_county[species_by_county > upper_fence]

top15 = species_by_county.head(15)
plt.figure()
top15.plot(kind='bar')
plt.title("Top 15 NY Counties by Unique Species (Richness)")
plt.xlabel("County")
plt.ylabel("Unique species count")
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

print({"n_outlier_counties_by_richness": int(outliers.shape[0])})
outliers.head(10)



### Method 2 — Correlation Analysis  
**Question:** Are **rare species** concentrated where **total species** are high?  
**Method:** Pearson correlation (county-level).  
**Plot:** Scatter with fitted regression line (legend-labeled).


In [ ]:

rare_mask = df_unique['state_rarity_score'] >= 4  # S1–S2, SH/SX
rare_counts = df_unique[rare_mask].groupby('county')['scientific_name'].nunique()

county_df = pd.DataFrame({
    'total_species': species_by_county,
    'rare_species': rare_counts
}).fillna(0)

r, p = stats.pearsonr(county_df['total_species'], county_df['rare_species'])
print({"pearson_r": r, "p_value": p})

x = county_df['total_species'].values
y = county_df['rare_species'].values
slope, intercept, r_value, p_value, std_err = stats.linregress(x, y)

plt.figure()
plt.scatter(x, y, label="County")
x_line = np.linspace(x.min(), x.max(), 100)
y_line = slope * x_line + intercept
plt.plot(x_line, y_line, label="OLS fit")
plt.title("Total vs. Rare Species by County")
plt.xlabel("Total unique species")
plt.ylabel("Rare species (S1–S2/SH/SX)")
plt.legend()
plt.tight_layout()
plt.show()



### Method 3 — Hypothesis Testing (Required)  
**Question:** Do **Animals** and **Plants** have the **same rarity distribution**?  
**Method:** Mann–Whitney U (non-parametric; compares medians/distributions).  
**Plot:** Boxplot of `state_rarity_score` by category.


In [ ]:

a = df_unique[df_unique['category'].str.upper()=='ANIMAL']['state_rarity_score'].dropna()
b = df_unique[df_unique['category'].str.upper()=='PLANT']['state_rarity_score'].dropna()

print({"n_animals": len(a), "n_plants": len(b)})
U, pval = stats.mannwhitneyu(a, b, alternative='two-sided')
print({"test": "Mann-Whitney U", "U": U, "p_value": pval})

plt.figure()
plt.boxplot([a.values, b.values], labels=['Animal', 'Plant'])
plt.title("State Rarity Score by Category")
plt.ylabel("Rarity score (5 rare … 1 common)")
plt.tight_layout()
plt.show()



## 3) Three Conclusions

**Conclusion 1 (Method 1 — Composition & Outliers):**  
Category composition is dominated by the most common class(es), and county **species richness** is **highly heterogeneous** with a small set of **outlier counties** (above the IQR upper fence), indicating uneven habitat diversity and/or survey effort.

**Conclusion 2 (Method 2 — Correlation):**  
There is a **strong positive correlation** between **total species** and **rare species** per county (high Pearson *r* with statistically significant *p*), implying that conserving species-rich counties protects a disproportionate share of rare taxa.

**Conclusion 3 (Method 3 — Hypothesis Test):**  
The **rarity score distributions** for **Animals** vs **Plants** are **significantly different** (Mann–Whitney U, *p* < 0.05), suggesting category-specific conservation strategies may be warranted.
